In [1]:
!pip install torch torchvision

In [2]:
!pip install lava

In [3]:
!pip install lava-dl

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [12]:
import torch
from torchvision import datasets, transforms

import logging
import numpy as np
import matplotlib.pyplot as plt

from lava.magma.core.run_configs import Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
from lava.utils.profiler import Profiler
import h5py
import lava.lib.dl.slayer as slayer
from lava.lib.dl.slayer import neuron
from lava.lib.dl.slayer import block

np.set_printoptions(linewidth=110)  # Increase the line lenght of output cells.

In [13]:
class Network(torch.nn.Module):
    def __init__(self):
        ...
        self.blocks = torch.nn.ModuleList(
            [  # sequential network blocks
                slayer.block.sigma_delta.Input(sdnn_params),
                slayer.block.sigma_delta.Conv(sdnn_params, 3, 24, 3),
                slayer.block.sigma_delta.Conv(sdnn_params, 24, 36, 3),
                slayer.block.rf_iz.Conv(rf_params, 36, 64, 3, delay=True),
                slayer.block.rf_iz.Conv(sdnn_cnn_params, 64, 64, 3, delay=True),
                slayer.block.rf_iz.Flatten(),
                slayer.block.alif.Dense(alif_params, 64 * 40, 100, delay=True),
                slayer.block.cuba.Recurrent(cuba_params, 100, 50),
                slayer.block.cuba.KWTA(cuba_params, 50, 50, num_winners=5),
            ]
        )

    def forward(self, x):
        for block in self.blocks:
            # forward computation is as simple as calling the blocks in a loop
            x = block(x)
        return x

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, "w")
        layer = h.create_group("layer")
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f"{i}"))

In [10]:
class LoihiNetwork(slayer.Network):
    def __init__(self):
        super(LoihiNetwork, self).__init__()

        # Define neuron parameters
        neuron_params = {
            'threshold'     : 1.25,
            'current_decay' : 0.25,
            'voltage_decay' : 0.03,
            'tau_grad'      : 0.03,
            'scale_grad'    : 3,
            'requires_grad' : True,
        }
        
        # For dropout, this might require custom implementation in Lava
        neuron_params_drop = {**neuron_params, 'dropout_prob': 0.05}

        # Define the network blocks with dropout and delay
        self.blocks = [
            block.Dense(neuron_params_drop, 128*128*2, 512, weight_norm=True, delay=True),
            block.Dense(neuron_params_drop, 512, 512, weight_norm=True, delay=True),
            block.Dense(neuron_params, 512, 10, weight_norm=True),
        ]

    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike

    def export_hdf5(self, filename):
        # Network export to hdf5 format
        with h5py.File(filename, 'w') as h:
            layer_group = h.create_group('layer')
            for i, block in enumerate(self.blocks):
                block_group = layer_group.create_group(f'{i}')
                # Export block parameters to HDF5
                # This will require specific implementation based on Lava's API

# Example usage
network = LoihiNetwork()
network.export_hdf5('network.hdf5')


AttributeError: module 'lava.lib.dl.slayer' has no attribute 'Network'

In [ ]:
import torch
val_path = 'final_project/val_dataset.pth'
train_path = 'final_project/train_dataset.pth'
# Load the model
test_val_dataset_loaded= torch.load(val_path)
train_dataset_loaded= torch.load(train_path)

In [ ]:
import tqdm
import math
import numpy as np
import torch

def split_to_train_test_set(train_ratio: float, origin_dataset, num_classes: int, random_split: bool = False):
    '''
    Splits the original dataset into train and test sets for Loihi.

    :param train_ratio: Ratio of the original dataset to be used as the train set.
    :param origin_dataset: The original dataset, preprocessed for SNNs if necessary.
    :param num_classes: Total number of classes, e.g., 10 for the MNIST dataset.
    :param random_split: If True, randomly split samples in each class.
    :return: Tuple (train_set, test_set) suitable for training/testing on Loihi.
    '''
    label_idx = [[] for _ in range(num_classes)]

    for i, item in enumerate(tqdm.tqdm(origin_dataset)):
        y = item[1]
        y = y.item() if isinstance(y, (np.ndarray, torch.Tensor)) else y
        label_idx[y].append(i)

    train_idx, test_idx = [], []
    if random_split:
        for labels in label_idx:
            np.random.shuffle(labels)

    for labels in label_idx:
        pos = math.ceil(len(labels) * train_ratio)
        train_idx.extend(labels[:pos])
        test_idx.extend(labels[pos:])

    # Note: Ensure the dataset is in a format suitable for SNNs here
    return torch.utils.data.Subset(origin_dataset, train_idx), torch.utils.data.Subset(origin_dataset, test_idx)

# Example usage
# origin_dataset = <your_dataset_preprocessed_for_SNN>
# train_set, test_set = split_to_train_test_set_loihi(0.8, origin_dataset, 10, random_split=True)


In [ ]:
def load_npz_frames(file_name: str) -> np.ndarray:
    '''
    :param file_name: path of the npz file that saves the frames
    :type file_name: str
    :return: frames
    :rtype: np.ndarray
    '''
    return np.load(file_name, allow_pickle=True)['frames'].astype(np.float32)

In [ ]:
import torchvision
test_ds = torchvision.datasets.DatasetFolder('content/CIFAR/frames_number_10_split_by_number', loader = load_npz_frames, extensions=('.npz', '.npy'))#, data_type = 'frame', frames_number = 10, split_by = 'number')

In [ ]:
training_set, testing_validation_set = split_to_train_test_set(0.7, test_ds, 10)

In [ ]:
import os
import time
from torch.utils.data import DataLoader, random_split
from lava.lib.dl.slayer import loihi as slayer  # Import Lava's Loihi-compatible modules

trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)

# Initialize the network for Loihi
start_time = time.time()
net = LoihiNetwork()  # This is your SNN defined for Loihi
end_time = time.time()
setup_time = end_time - start_time
print(f"Setup time: {setup_time:.4f} seconds")

# Data preprocessing for SNNs
# Assuming train_dataset_loaded and test_val_dataset_loaded are preprocessed for SNNs
training_set = train_dataset_loaded
testing_validation_set = test_val_dataset_loaded

subset_size = len(testing_validation_set)
subset1_size = subset_size // 2
subset2_size = subset_size - subset1_size

# Use random_split to split the initial subset randomly
val_set, testing_set = random_split(testing_validation_set, [subset1_size, subset2_size])

# Create DataLoaders for spike-encoded data
train_loader = DataLoader(dataset=training_set, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=testing_set, batch_size=16, shuffle=True)

In [ ]:
error = slayer.loss.SpikeRate(true_rate=0.2, false_rate=0.03, reduction='sum').to(device)
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats, classifier=slayer.classifier.Rate.predict)